In [118]:
import pandas as pd
import numpy as np

In [119]:
d = pd.read_csv('../tables/merged_final_non_org_fixed.csv', parse_dates=['Time_Mid'])

In [120]:
bin_labels = ['SMPS_Bin02', 'SMPS_Bin03', 'SMPS_Bin04', 'SMPS_Bin05',
       'SMPS_Bin06', 'SMPS_Bin07', 'SMPS_Bin08', 'SMPS_Bin09', 'SMPS_Bin10',
       'SMPS_Bin11', 'SMPS_Bin12', 'SMPS_Bin13', 'SMPS_Bin14', 'SMPS_Bin15',
       'SMPS_Bin16', 'SMPS_Bin17', 'SMPS_Bin18', 'SMPS_Bin19', 'SMPS_Bin20',
       'SMPS_Bin21', 'SMPS_Bin22', 'SMPS_Bin23', 'SMPS_Bin24', 'SMPS_Bin25',
       'SMPS_Bin26', 'SMPS_Bin27', 'SMPS_Bin28', 'SMPS_Bin29', 'SMPS_Bin30', 'LAS_Bin01', 'LAS_Bin02', 'LAS_Bin03', 'LAS_Bin04', 'LAS_Bin05',
       'LAS_Bin06', 'LAS_Bin07', 'LAS_Bin08', 'LAS_Bin09', 'LAS_Bin10',
       'LAS_Bin11', 'LAS_Bin12', 'LAS_Bin13', 'LAS_Bin14', 'LAS_Bin15',
       'LAS_Bin16', 'LAS_Bin17', 'LAS_Bin18', 'LAS_Bin19', 'LAS_Bin20',
       'LAS_Bin21', 'LAS_Bin22', 'LAS_Bin23', 'LAS_Bin24', 'LAS_Bin25',
       'LAS_Bin26']

lower_bounds = [3.36, 3.76, 4.22, 4.74, 5.32, 5.97, 6.69, 7.51, 8.43, 9.46, 10.6, 11.9, 13.4, 15.0, 16.8, 18.9, 21.2, 23.8, 26.7, 29.9, 33.6, 37.6, 42.2, 47.4, 53.2, 59.7, 66.9, 75.1, 84.3, 93.9, 106.1, 119.0, 133.6, 149.9, 168.2, 188.7, 211.7, 237.5, 266.5, 299.0, 335.5, 376.5, 422.4, 473.9, 531.8, 596.6, 669.5, 751.1, 842.8, 945.6, 1129.5, 1421.9, 1790.1, 2253.6, 2837.1]
upper_bounds = [3.76, 4.22, 4.74, 5.32, 5.97, 6.69, 7.51, 8.43, 9.46, 10.6, 11.9, 13.4, 15.0, 16.8, 18.9, 21.2, 23.8, 26.7, 29.9, 33.6, 37.6, 42.2, 47.4, 53.2, 59.7, 66.9, 75.1, 84.3, 94.0, 106.1, 119.0, 133.6, 149.9, 168.2, 188.7, 211.7, 237.5, 266.5, 299.0, 335.5, 376.5, 422.4, 473.9, 531.8, 596.6, 669.5, 751.1, 842.8, 945.6, 1129.5, 1421.9, 1790.1, 2253.6, 2837.1, 3487.5]

dlogd=[]
for i in range(0,len(lower_bounds)):
       dlogd.append(np.log10(upper_bounds[i]) - np.log10(lower_bounds[i]))

d['N_tot'] = (d[bin_labels] * dlogd).sum(axis=1)
d['OMF'] = d['Org_Ave_IsoK_STP'] / d['ams_tot']
d['f_SMPS_LAS'] = d[bin_labels].notna().mean(axis=1)

In [121]:
d = d[d['f_SMPS_LAS'] > .95]

In [122]:
d = d.dropna(subset=['N_CCN_stdPT', 'N_tot', 'D_a', 'k_obs', 'OMF'])

In [123]:
transit_flights = [pd.Timestamp('2022-03-22').date(), pd.Timestamp('2022-05-18').date(), pd.Timestamp('2022-05-21').date(), pd.Timestamp('2022-05-31').date(), pd.Timestamp('2022-06-18').date()]
df_winter = d[d['Time_Mid'].dt.month.isin([11, 12, 1, 2, 3, 4]) & (~d['Time_Mid'].dt.date.isin(transit_flights))]
df_spring = d[d['Time_Mid'].dt.month.isin([5, 6]) & (np.logical_not((d['Time_Mid'].dt.month.isin([6])) & (d['Time_Mid'].dt.year.isin([2022])))) & (~d['Time_Mid'].dt.date.isin(transit_flights))]
df_summer = d[d['Time_Mid'].dt.month.isin([8, 9]) & (~d['Time_Mid'].dt.date.isin(transit_flights))]
df_bermuda = d[(d['Time_Mid'].dt.month.isin([6])) & (d['Time_Mid'].dt.year.isin([2022])) & (~d['Time_Mid'].dt.date.isin(transit_flights))]

In [124]:
ds = [df_winter, df_spring, df_summer, df_bermuda]

In [125]:
d_out = pd.DataFrame(columns = ['N', 'N_CCN', 'N_tot', 'D_c', 'k_obs', 'OMF'])

In [126]:
def print_cell(d, dgts=0):
    return f"{d.median():.{dgts}f} ({d.quantile(.25):.{dgts}f}, {d.quantile(.75):.{dgts}f})"

In [127]:
for i in range(0,4):
    d_out.loc[i,'N'] = ds[i].shape[0]
    d_out.loc[i,'N_CCN'] = print_cell(ds[i]['N_CCN_stdPT'])
    d_out.loc[i,'N_tot'] = print_cell(ds[i]['N_tot'])
    d_out.loc[i,'D_c'] = print_cell(ds[i]['D_a'], 1)
    d_out.loc[i,'k_obs'] = print_cell(ds[i]['k_obs'], 2)
    d_out.loc[i,'OMF'] = print_cell(ds[i]['OMF'], 2)

In [128]:
d_out

,N,N_CCN,N_tot,D_c,k_obs,OMF
0,5757,"475 (235, 826)","1714 (868, 3512)","74.9 (63.5, 89.2)","0.30 (0.17, 0.48)","0.46 (0.35, 0.58)"
1,3851,"512 (284, 974)","1104 (576, 2232)","76.2 (62.1, 86.0)","0.27 (0.19, 0.46)","0.58 (0.34, 0.73)"
2,472,"744 (275, 1262)","1686 (1095, 2261)","77.9 (58.7, 123.4)","0.24 (0.14, 0.36)","0.66 (0.59, 0.74)"
3,1542,"357 (268, 495)","591 (426, 956)","49.1 (42.3, 62.0)","0.74 (0.25, 1.41)","0.35 (0.20, 0.56)"


In [131]:
d_out.to_excel('../tables/summary_stats.xlsx', index=False)